# Detect Cut in the video

In [1]:
from scenedetect import detect, ContentDetector,ThresholdDetector,AdaptiveDetector,open_video,StatsManager,SceneManager
import numpy as np
from MyDetector import MyDetector as MyDetector1

/home/guangyi.liu/.conda/envs/lgy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [80]:
import os
# root_path = '/home/guangyi.liu/shared-folder/datasets/guangyi_data/kinetics-dataset/k600/subset'  # for cut-video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/cut'  # for non-cut video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/noncut'  # for non-cut video
# root_path = '/home/guangyi.liu/shared-folder/datasets/guangyi_data/scripts/panda70m/samples_fewer'
# root_path = '/home/guangyi.liu/repo/video_process/mp4_file'
root_path ='/home/guangyi.liu/shared-folder/datasets/panda-oricut-500k-720p-clips/debug'
files = os.listdir(root_path)
files.sort()
cut_list = []
static_list =[]
results_list = {'mp4':[],'de_mean':[],'de_std':[],'dl_mean':[],'dl_std':[]}
for mp4_file in files:
    if '.mp4' in mp4_file[-4:] and '22' in mp4_file:
        # print(mp4_file)
        mp4file = os.path.join(root_path, mp4_file)
        video = open_video(mp4file)
        video.seek(1)
        stats_manager = StatsManager()
        scene_manager = SceneManager(stats_manager)
        # scene_manager.auto_downscale=True
        # scene_manager.downscale = 1
        scene_manager.add_detector(ContentDetector(threshold=27))
        
        scene_manager.detect_scenes(video=video)
        stats_manager._frame_metrics
        frame_keys = stats_manager._frame_metrics.keys()
        delta_edges = []
        delta_lums = []
        for key in frame_keys:
            delta_edges.append(stats_manager._frame_metrics[key]['delta_edges'])
            delta_lums.append(stats_manager._frame_metrics[key]['delta_lum'])
        de_mean = np.mean(delta_edges)
        de_std = np.std(delta_edges)
        dl_mean = np.mean(delta_lums)
        dl_std = np.std(delta_lums)
        de_upper_bound = max(de_mean + 4.7 * de_std, 3.* de_mean)
        dl_upper_bound = max(dl_mean + 5* dl_std, 3.* dl_mean)
        total_cut_de = np.sum(np.array(delta_edges) > de_upper_bound )
        total_cut_dl = np.sum(np.array(delta_lums) > dl_upper_bound   )
        results_list['mp4'].append(mp4_file)
        results_list['de_mean'].append(round(de_mean,2))
        results_list['dl_mean'].append(round(dl_mean,2))
        results_list['de_std'].append(round(de_std,2))
        results_list['dl_std'].append(round(dl_std,2))
        print(mp4_file,round(de_mean,2),round(de_std,2),round(de_upper_bound,2),round(np.max(delta_edges),2),'\t',round(dl_mean,2),round(dl_std,2),round(dl_upper_bound),round(np.max(delta_lums),2))
        if dl_mean < .5 and dl_std < 0.5 :
            static_list.append(mp4_file)
        elif de_mean < 0.6 and de_std < 0.6:
            static_list.append(mp4_file)
        if total_cut_de> 0 or total_cut_dl >0 or (dl_mean > 9 and dl_std>9) :
            cut_list.append(mp4_file)
        stats_manager.save_to_csv(f'./mp4_file/{mp4_file}.csv')
        # print(mp4_file,total_cut>0)
        # if mp4_file =='4.6.mp4':
        #     break
        # [3.99,4.03,4.12,4.27,4.53,4.8,4.81,4.97,5.07]
# 15 ['3.72.mp4', '3.83.mp4',  '4.37.mp4', '4.6.mp4',  '4.83.mp4', '4.86.mp4']
print(len(cut_list),cut_list)

220dTSMLdj0_0.mp4 15.05 11.23 67.85 86.01 	 6.69 5.91 36 55.82
220dTSMLdj0_1.mp4 5.86 1.61 17.57 9.44 	 2.18 0.57 7 3.61
220dTSMLdj0_2.mp4 12.8 8.04 50.6 90.84 	 3.94 3.08 19 38.36
221TWSrSk-Q_1.mp4 12.3 7.63 48.14 32.08 	 3.26 2.33 15 8.8
226uOn6khAs_2.mp4 13.33 15.61 86.68 104.72 	 5.39 9.8 54 70.8
227ITymDcA8_2.mp4 3.71 1.67 11.53 8.61 	 0.83 0.51 3 2.26
22A5vfSCbEw_2.mp4 33.06 16.02 108.38 108.13 	 13.04 6.79 47 43.03
22ExhAyUNFQ_0.mp4 16.09 7.54 51.52 53.97 	 4.99 3.66 23 30.94
22ExhAyUNFQ_1.mp4 4.41 1.85 13.23 11.44 	 2.64 1.31 9 6.62
22GRZ7ih-mQ_0.mp4 6.34 2.92 20.08 18.72 	 0.85 0.91 5 5.04
22GRZ7ih-mQ_2.mp4 0.04 0.11 0.55 1.29 	 0.09 0.1 1 1.22
22I2D9WDIHQ_0.mp4 6.1 4.32 26.42 54.13 	 2.65 2.58 16 38.55
22I2D9WDIHQ_2.mp4 6.18 7.09 39.52 95.39 	 2.23 4.34 24 57.48
22IdOHUBAys_0.mp4 3.87 2.42 15.24 11.89 	 1.6 0.98 7 4.6
22IdOHUBAys_1.mp4 10.85 9.08 53.55 105.31 	 6.92 6.59 40 60.7
22IdOHUBAys_2.mp4 6.47 1.61 19.4 9.4 	 2.2 0.71 7 3.33
22Nc43oY0Ng_0.mp4 3.54 1.07 10.63 6.7 	 1.0

In [73]:
cut_=set(['1179HKeCzdM_0.mp4', '1179HKeCzdM_1.mp4', '117hrav2N3E_0.mp4','11DFBNgL5lc_0.mp4','11I5Qvkl3CQ_0.mp4',
      '11Lw4XsJ-iQ_0.mp4','11Lw4XsJ-iQ_2.mp4','11BGn_PzXz8_0.mp4','11HjRq-A3ow_2.mp4','11NM_Obu9yw_1.mp4',
      '11YUkJCf6_w_1.mp4','11fLImeUZ2w_2.mp4', '11igmz4iZrg_2.mp4','11jK-vunBW4_0.mp4','11ogjdyiMwQ_2.mp4',
     '11uM9TnvFmA_0.mp4', '11uM9TnvFmA_1.mp4','11bnBlhjw9Y_1.mp4','11VlmfAJEyY_0.mp4','11xJjSP9psQ_2.mp4',
         '11bnBlhjw9Y_0.mp4'])
nocut_=set(['11DFBNgL5lc_1.mp4','11GLHBfciaA_2.mp4','11HXe7pymh8_2.mp4',
 '11I5Qvkl3CQ_1.mp4','114iopCPWKk_0.mp4','11508ex1Xxg_0.mp4','11DFBNgL5lc_2.mp4','11GLHBfciaA_2.mp4',
       '11HXe7pymh8_0.mp4', '11HXe7pymh8_1.mp4','11HjRq-A3ow_1.mp4','11I5Qvkl3CQ_1.mp4',])

 

In [81]:
static_list

['22GRZ7ih-mQ_2.mp4', '22Nc43oY0Ng_2.mp4', '22pBN04LBz4_2.mp4']

In [75]:
set(results_list['mp4'])- set(static_list) - set(cut_list)

{'114iopCPWKk_0.mp4',
 '11DFBNgL5lc_2.mp4',
 '11GLHBfciaA_2.mp4',
 '11HXe7pymh8_0.mp4',
 '11HXe7pymh8_1.mp4',
 '11HXe7pymh8_2.mp4',
 '11HjRq-A3ow_1.mp4',
 '11YUkJCf6_w_2.mp4',
 '11fLImeUZ2w_0.mp4',
 '11fLImeUZ2w_1.mp4',
 '11igmz4iZrg_1.mp4',
 '11mhs3qdQk0_2.mp4',
 '11ssCPxWDaY_2.mp4',
 '11vajpsGDMc_1.mp4'}

In [67]:
cut_cnt=0
no_cut_cnt = 0
len_nocut = len(nocut_)
len_cut = len(cut_)
for cut_mp4 in cut_:
    if cut_mp4 in cut_list:
        cut_cnt+=1
print(f'cut: {cut_cnt}/{len_cut}')

for no_cut_mp4 in nocut_:
    if no_cut_mp4 in cut_list:
        no_cut_cnt+=1
        print(no_cut_mp4)
print(f'correct no cut: {len_nocut-no_cut_cnt}/{len_nocut}')

cut: 21/21
11I5Qvkl3CQ_1.mp4
11DFBNgL5lc_1.mp4
correct no cut: 8/10


In [13]:
import os
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes'  # for cut-video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/cut'  # for non-cut video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/noncut'  # for non-cut video
# root_path = '/home/guangyi.liu/shared-folder/datasets/guangyi_data/scripts/panda70m/samples_fewer'
# root_path = '/home/guangyi.liu/repo/video_process/mp4_file'
root_path ='/home/guangyi.liu/shared-folder/datasets/guangyi_data/MiraData'
files = os.listdir(root_path)
files.sort()
cut_list = []
static_list =[]
# results_list = {'mp4':[],'de_mean':[],'de_std':[],'dl_mean':[],'dl_std':[]}
for mp4_file in files:
    if '.mp4' in mp4_file[-4:]:
        # print(mp4_file)
        mp4file = os.path.join(root_path, mp4_file)
        video = open_video(mp4file)
        video.seek(1)
        stats_manager = StatsManager()
        scene_manager = SceneManager(stats_manager)
        # scene_manager.auto_downscale=True
        # scene_manager.downscale = 1
        scene_manager.add_detector(MyDetector1(threshold=[61.66, 38.847722996346846]))
        
        scene_manager.detect_scenes(video=video)
        stats_manager._frame_metrics
        frame_keys = stats_manager._frame_metrics.keys()
        # delta_edges = []
        # delta_lums = []
        # for key in frame_keys:
        #     delta_edges.append(stats_manager._frame_metrics[key]['delta_edges'])
        #     delta_lums.append(stats_manager._frame_metrics[key]['delta_lum'])
        # de_mean = np.mean(delta_edges)
        # de_std = np.std(delta_edges)
        # dl_mean = np.mean(delta_lums)
        # dl_std = np.std(delta_lums)
        # de_upper_bound = max(de_mean + 5.5 * de_std, 3* de_mean)
        # dl_upper_bound = max(dl_mean + 5.5* dl_std, 3* dl_mean)
        # total_cut_de = np.sum(np.array(delta_edges) > de_upper_bound )
        # total_cut_dl = np.sum(np.array(delta_lums) > dl_upper_bound   )
        # results_list['mp4'].append(mp4_file)
        # results_list['de_mean'].append(round(de_mean,2))
        # results_list['dl_mean'].append(round(dl_mean,2))
        # results_list['de_std'].append(round(de_std,2))
        # results_list['dl_std'].append(round(dl_std,2))
        # print(mp4_file,round(de_mean,2),round(de_std,2),de_upper_bound,round(np.max(delta_edges),2),'\t',round(dl_mean,2),round(dl_std,2),dl_upper_bound,round(np.max(delta_lums),2))
        # if dl_mean < 0.7 and dl_std < 0.3:
        #     static_list.append(mp4_file)
        # elif de_mean < 1.5 and de_std < 1.5:
        #     static_list.append(mp4_file)
        # if total_cut_de> 0 or total_cut_dl >0:
        #     cut_list.append(mp4_file)
        # stats_manager.save_to_csv(f'./mp4_file/{mp4_file}.csv')
        # print(mp4_file,total_cut>0)
        # if mp4_file =='4.6.mp4':
        #     break
        # [3.99,4.03,4.12,4.27,4.53,4.8,4.81,4.97,5.07]
# 15 ['3.72.mp4', '3.83.mp4',  '4.37.mp4', '4.6.mp4',  '4.83.mp4', '4.86.mp4']
print(len(cut_list),cut_list)

[61.66, 38.847722996346846]
0 []


In [14]:
scene_list = scene_manager.get_scene_list()

In [24]:
scene_list[0][0]

00:00:00.033 [frame=1, fps=29.970]

In [14]:
cut_flag = (np.array(delta_edges) > de_upper_bound) | (np.array(delta_lums) > dl_upper_bound)
cut_pos = []
for i in range(len(cut_flag)):
    if cut_flag[i]:
        cut_pos.append(i)

In [76]:
from datetime import datetime,timedelta
skip_duration = timedelta(seconds=0.5)
start_skip = timedelta(seconds=1.5)
video_name = 'test_new'
time_format = '%H:%M:%S.%f'
for i, timecode_ in enumerate(scene_list):
    timecode = [None,None]
    timecode[0] = timecode_[0].get_timecode()
    timecode[1] = timecode_[1].get_timecode()
    if i == 0:
        start_time = datetime.strptime(timecode[0], time_format) + skip_duration + start_skip
    else:
        start_time = datetime.strptime(timecode[0], time_format) + skip_duration
    end_time = datetime.strptime(timecode[1], time_format) - skip_duration
    timecode[0] = start_time.strftime(time_format)
    video_duration = (end_time - start_time).total_seconds()
    if video_duration < 2:
        print('skip')
        continue
    os.system("ffmpeg -hide_banner -loglevel panic -ss %s -t %.3f -i %s %s"%(timecode[0], video_duration, video_path, os.path.join('mp4_file', video_name+".%i.mp4"%i)))


skip
skip
skip
skip
skip
skip
skip
skip
skip


In [61]:
test= datetime.strptime(timecode[0], '%H:%M:%S.%f') + skip_duration

In [71]:
time_format = '%H:%M:%S.%f'

In [72]:
test.strftime(time_format)

'01:02:20.040020'

In [53]:
one_second

datetime.timedelta(microseconds=20)

In [38]:
stats_manager.get_metrics(2,['delta_edges'])

[28.457845052083332]

In [51]:
dl_mean + 5.5 * dl_std

79.91106163101028

In [41]:
stats_manager.save_to_csv(

['4.12.mp4', '4.5.mp4', '4.76.mp4', '4.77.mp4', '5.27.mp4']

In [32]:
for i in range(len(results_list['mp4'])):
    if results_list['de_mean'][i] < 1.5:
        print(results_list['mp4'][i], results_list['de_mean'][i],results_list['de_std'][i])

4.12.mp4 0.23 1.14
4.5.mp4 1.29 1.03
4.76.mp4 1.35 0.41
4.77.mp4 1.04 0.86


In [31]:
for i in range(len(results_list['mp4'])):
    if results_list['dl_mean'][i] < 1.:
        print(results_list['mp4'][i], results_list['dl_mean'][i],results_list['dl_std'][i] )

4.12.mp4 0.06 0.31
4.5.mp4 0.29 0.24
4.6.mp4 0.67 1.23
4.76.mp4 0.58 0.18
4.77.mp4 0.14 0.04
4.83.mp4 0.65 0.45
5.27.mp4 0.55 0.19


In [35]:
for i in range(len(results_list['mp4'])):
    if results_list['dl_mean'][i] > results_list['dl_std'][i] and results_list['dl_mean'][i] < 1.:
        print(results_list['mp4'][i], results_list['dl_mean'][i],results_list['dl_std'][i] )

4.5.mp4 0.29 0.24
4.76.mp4 0.58 0.18
4.77.mp4 0.14 0.04
4.83.mp4 0.65 0.45
5.27.mp4 0.55 0.19


In [27]:
results_list['dl_mean'][-1]

0.55

In [2]:
from scenedetect import detect, ContentDetector,open_video,StatsManager,SceneManager
import numpy as np
def detect_cut(mp4file):
    video = open_video(mp4file)
    video.seek(1) #  the first frame may give large delta_edge value, start from the second frame
    stats_manager = StatsManager()
    scene_manager = SceneManager(stats_manager)
    scene_manager.add_detector(ContentDetector())
    scene_manager.detect_scenes(video=video)
    frame_keys = stats_manager._frame_metrics.keys()  # read metrics
    delta_edges = []
    delta_lums = []
    for key in frame_keys:
        delta_edges.append(stats_manager._frame_metrics[key]['delta_edges'])
        delta_lums.append(stats_manager._frame_metrics[key]['delta_lum'])
    de_mean = np.mean(delta_edges)
    de_std = np.std(delta_edges)
    dl_mean = np.mean(delta_lums)
    dl_std = np.std(delta_lums)
    de_upper_bound = max(de_mean + 5.5 * de_std, 3* de_mean)
    dl_upper_bound = max(dl_mean + 5.5 * dl_std, 3* dl_mean)
    total_cut_de = np.sum(np.array(delta_edges) > de_upper_bound )
    total_cut_dl = np.sum(np.array(delta_lums) > dl_upper_bound )
    return total_cut_de> 0 or total_cut_dl >0 # if True, there is a cut, ignore this sample

/home/guangyi.liu/.conda/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import os
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes'  # for cut-video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/cut'  # for non-cut video
# root_path = '/home/guangyi.liu/world_model/ipynb/panda_samples_aes/noncut'  # for non-cut video
root_path = '/home/guangyi.liu/shared-folder/datasets/guangyi_data/kinetics-dataset/k600_targz/train/bobsledding'
files = os.listdir(root_path)
files.sort()
cut_list = []
import time
start = time.time()
from tqdm import tqdm
out_list=[]
for mp4_file in tqdm(files,total=len(files)):
    if '.mp4' in mp4_file:
        # print(mp4_file)
        mp4file = os.path.join(root_path, mp4_file)
        res = detect_cut(mp4file)
        if res:
            out_list.append(mp4_file)
        #     print(mp4_file)

print(time.time()-start)
# [3.99,4.03,4.12,4.27,4.53,4.8,4.81,4.97,5.07]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 550/550 [03:43<00:00,  2.46it/s]

223.59946084022522


In [4]:
from streaming import StreamingDataset

In [11]:
print(len(out_list)/len(files))

0.33454545454545453


In [5]:
dataset = StreamingDataset(remote = '/home/guangyi.liu/shared-folder/datasets/panda70m_streaming_all_2s_10s_2xframe_ofm1.5_cutnum_aes4.75_forcaption_gpt_v3_cut')

In [6]:
len(dataset)

627481